In [1]:
import os
%pwd

'f:\\ml projects\\brain_tumor_classification\\notebooks'

In [2]:
os.chdir("../")
%pwd

'f:\\ml projects\\brain_tumor_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluateConfig:
    path_of_model:Path
    test_data_path:Path
    params_batch_size:int
    params_image_size:list

In [4]:
from brain_tumor_classification.constants import *
from brain_tumor_classification.utils.common import create_directories,read_yaml
import os

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILEPATH,params_filepath = PARAMS_FILEPATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluate_config(self) -> ModelEvaluateConfig:

        test_data_path = os.path.join(self.config.data_ingestion.unzip_dir,"brain_tumor_mris","Testing")
        model_evaluate_config = ModelEvaluateConfig(
            path_of_model= Path(self.config.model_training.trained_model_path),
            test_data_path= Path(test_data_path),
            params_batch_size= self.params.BATCH_SIZE,
            params_image_size= self.params.IMAGE_SIZE
        )
        return model_evaluate_config

In [6]:
import tensorflow as tf
from brain_tumor_classification.utils.common import save_json

In [13]:
class ModelEvaluate:
    def __init__(self,config:ModelEvaluateConfig) -> None:
        self.config = config

    @property
    def load_model(self) -> tf.keras.models:
        return tf.keras.models.load_model(self.config.path_of_model)
    
    def train_generator(self):
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale = 1./255
        )
        test_data = test_datagenerator.flow_from_directory(
            directory = self.config.test_data_path,
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            class_mode = "categorical"
        )
        return test_data
    
    def evaluate(self):
        self.model = self.load_model
        test_data = self.train_generator()
        self.score = self.model.evaluate(test_data)

    def save_score(self):
        score = {"Loss":self.score[0],"Accuracy":self.score[1]}
        save_json(path = Path("scores.json"),data = score)



In [14]:
try:
    config = ConfigurationManager()
    model_evaluate_config = config.get_model_evaluate_config()
    model_evaluate = ModelEvaluate(config=model_evaluate_config)
    model_evaluate.evaluate()
    model_evaluate.save_score()
except Exception as e:
    raise e

Found 576 images belonging to 4 classes.

18/18 [==============================] - 350s 19s/step - loss: 0.1703 - accuracy: 0.9323
